<a href="https://colab.research.google.com/github/schumbar/CMPE258/blob/chumbar%2Fassignment_03/assignment_03/CMPE258_assignment03_part_C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Assignment 03 Part C - Keras-NLP

## Assignment Description
Use Keras-NLP for various NLP tasks. Perform the following tasks:

- Inference with a pretrained classifier
- Fine-tune a pretrained backbone.
- Fine-tune with user-controlled pre-processing.
- Fine-tune a custom model.

## References Used
1. [Getting Started with KerasNLP](https://colab.research.google.com/github/keras-team/keras-io/blob/master/guides/ipynb/keras_nlp/getting_started.ipynb)
2. ChatGPT was used for general debugging purposes.

## Setup

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install -q --upgrade keras
!pip install -q transformers
!pip install -q --upgrade tensorflow
!pip install -q --upgrade keras-nlp
!pip install -q keras-nlp

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 14.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 286.8/286.8 kB 18.7 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.15.0 requires keras<2.16,>=2.15.0, but you have keras 3.1.1 which is incompatible.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 589.8/589.8 MB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 111.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 107.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 106.9 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tf-keras 2.15.1 requires tensorflow<2.16,>=2.15, but you have tensorfl

In [3]:
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from keras_nlp.models import BertClassifier
import tensorflow_text
from tensorflow import feature_column
import tensorflow
import pandas as pd
import numpy as np
import os
import keras_nlp
import keras


In [4]:
os.environ["KERAS_BACKEND"] = "tensorflow"
keras.mixed_precision.set_global_policy("mixed_float16")

## Inference with Pre-trained Model

In [5]:
# To speed up training and generation, we use preprocessor of length 128
# instead of full length 1024.
preprocessor = keras_nlp.models.GPT2CausalLMPreprocessor.from_preset(
    "gpt2_base_en",
    sequence_length=128,
)
gpt2_lm = keras_nlp.models.GPT2CausalLM.from_preset(
    "gpt2_base_en", preprocessor=preprocessor
)

100%|██████████| 448/448 [00:00<00:00, 692kB/s]
100%|██████████| 446k/446k [00:00<00:00, 997kB/s] 
100%|██████████| 0.99M/0.99M [00:00<00:00, 1.79MB/s]
100%|██████████| 484/484 [00:00<00:00, 664kB/s]
100%|██████████| 475M/475M [00:13<00:00, 37.5MB/s]


In [6]:
output = gpt2_lm.generate("My trip to Yosemite was", max_length=200)
print("\nGPT-2 output:")
print(output)


GPT-2 output:
My trip to Yosemite was a whirlwind. It was a beautiful day, with the sun setting and the snow melting away. I was so happy to be back in the park and to be able to enjoy the beautiful views.

I was able to spend a few minutes with the park staff, who were very helpful with my day and I had a nice chat. They had the opportunity to talk to me about some of the things I wanted to do in Yosemite in the future.

After my visit to Yosemite and the park was over, the staff gave me an opportunity to get a tour and a tour guide. I got to see a few things that I wanted to do in Yosemite in the future:

The Big Bear Trail, a great place to get a glimpse of what Yosemite can do. The Big Bear is a great place to see the Yosemite River. I was able to take a tour of the park and the Big Bear.

The Redwoods, a great place


## Dataset
Dataset Link: [IMDB Dataset of 50K Movie Reviews](https://www.kaggle.com/datasets/lakshmi25npathi/imdb-dataset-of-50k-movie-reviews)

In [7]:
file_path = '/content/drive/MyDrive/Spring2024/CMPE258/Assignment03/data/part_c/Processed_IMDB_Dataset.csv'

In [8]:
# Load the dataset
data = pd.read_csv(file_path)

# Display the first few rows of the dataset to understand its structure
data.head()

,review,sentiment
0,one of the other reviewers has mentioned that ...,1
1,a wonderful little production the filming tech...,1
2,i thought this was a wonderful way to spend ti...,1
3,basically theres a family where a little boy j...,0
4,petter matteis love in the time of money is a ...,1


In [9]:
test_input = data.loc[0,'review']
test_input

'one of the other reviewers has mentioned that after watching just  oz episode youll be hooked they are right as this is exactly what happened with methe first thing that struck me about oz was its brutality and unflinching scenes of violence which set in right from the word go trust me this is not a show for the faint hearted or timid this show pulls no punches with regards to drugs sex or violence its is hardcore in the classic use of the wordit is called oz as that is the nickname given to the oswald maximum security state penitentary it focuses mainly on emerald city an experimental section of the prison where all the cells have glass fronts and face inwards so privacy is not high on the agenda em city is home to manyaryans muslims gangstas latinos christians italians irish and moreso scuffles death stares dodgy dealings and shady agreements are never far awayi would say the main appeal of the show is due to the fact that it goes where other shows wouldnt dare forget pretty picture

#### Helper Functions

In [10]:
# Helper function to transform dataset to categorical format.
def transform_dataset_to_cateogrical_format(dataframe):
  X = dataframe.loc[:, 'review'].astype(str)
  y = dataframe.loc[:, 'sentiment']
  X_train, X_test , y_train, y_test = train_test_split(X, y, test_size = 0.20)
  X_train_tfdata = tensorflow.data.Dataset.from_tensor_slices(X_train.values)
  X_test_tfdata = tensorflow.data.Dataset.from_tensor_slices(X_test.values)
  train_ds = tensorflow.data.Dataset.zip((X_train_tfdata, tensorflow.data.Dataset.from_tensor_slices(y_train)))
  test_ds = tensorflow.data.Dataset.zip((X_test_tfdata, tensorflow.data.Dataset.from_tensor_slices(y_test)))
  return train_ds, test_ds

## Fine-tuning a pretrained backbone

In [11]:
classifier = keras_nlp.models.BertClassifier.from_preset("bert_tiny_en_uncased_sst2")

100%|██████████| 2.14k/2.14k [00:00<00:00, 2.69MB/s]
100%|██████████| 226k/226k [00:00<00:00, 693kB/s]
100%|██████████| 16.8M/16.8M [00:01<00:00, 15.8MB/s]
/usr/local/lib/python3.10/dist-packages/keras/src/saving/saving_lib.py:396: UserWarning: Skipping variable loading for optimizer 'loss_scale_optimizer', because it has 4 variables whereas the saved optimizer has 2 variables. 
  trackable.load_own_variables(weights_store.get(inner_path))
/usr/local/lib/python3.10/dist-packages/keras/src/saving/saving_lib.py:396: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 0 variables. 
  trackable.load_own_variables(weights_store.get(inner_path))


In [12]:
predictions = classifier.predict(data.loc[:,'review'])

1563/1563 ━━━━━━━━━━━━━━━━━━━━ 11s 6ms/step


In [13]:
def classifier_predict(prediction):
  if prediction[0] > prediction[1]:
    return 0
  return 1

In [14]:
sentiment_predictions = [classifier_predict(pred) for pred in predictions]
data['predicted_y'] = sentiment_predictions

In [15]:
data.head()

,review,sentiment,predicted_y
0,one of the other reviewers has mentioned that ...,1,0
1,a wonderful little production the filming tech...,1,1
2,i thought this was a wonderful way to spend ti...,1,1
3,basically theres a family where a little boy j...,0,0
4,petter matteis love in the time of money is a ...,1,1


## Fine tuning with user-controlled preprocessing

In [16]:
pretrained_classifier_from_modules = keras_nlp.models.BertClassifier.from_preset(
    "bert_tiny_en_uncased",
    num_classes=2
)

100%|██████████| 507/507 [00:00<00:00, 742kB/s]
100%|██████████| 16.8M/16.8M [00:01<00:00, 15.9MB/s]
100%|██████████| 547/547 [00:00<00:00, 675kB/s]
100%|██████████| 226k/226k [00:00<00:00, 689kB/s]


In [17]:
train_ds, test_ds = transform_dataset_to_cateogrical_format(data)

In [18]:
pretrained_classifier_from_modules.fit(
    train_ds.batch(batch_size=10),
    validation_data=test_ds.batch(batch_size=10),
    epochs=1,
)

4000/4000 ━━━━━━━━━━━━━━━━━━━━ 59s 9ms/step - loss: 0.4586 - sparse_categorical_accuracy: 0.7732 - val_loss: 0.2789 - val_sparse_categorical_accuracy: 0.8832


In [19]:
predictions = pretrained_classifier_from_modules.predict(test_ds.batch(batch_size=5))

2000/2000 ━━━━━━━━━━━━━━━━━━━━ 6s 2ms/step


In [20]:
module_sentiment_preds = [classifier_predict(pred) for pred in predictions]

In [21]:
pretrained_module_test = pd.DataFrame({'review': test_ds, 'predicted_y': module_sentiment_preds})

In [22]:
pretrained_module_test

,review,predicted_y
0,(tf.Tensor(b'i saw the original rough screen s...,1
1,(tf.Tensor(b'after the success of scoobydoo wh...,0
2,(tf.Tensor(b'hg is normally exploitive and its...,0
3,(tf.Tensor(b'this isin shortthe tv comedy seri...,1
4,(tf.Tensor(b'summer phoenix did a great perfor...,1
...,...,...
9995,(tf.Tensor(b'black comedy isnt always an easy ...,1
9996,(tf.Tensor(b'a young basketballplaying profess...,0
9997,(tf.Tensor(b'yet another remake of fistful of ...,1
9998,(tf.Tensor(b'ouch what a mess we have here not...,0


## Fine tuning a custom model

In [23]:
# Load your preprocessed data
df = pd.read_csv(file_path)  # Replace with your data path
reviews = df["review"]
sentiments = df["sentiment"]

In [24]:
preprocessor = keras_nlp.models.BertPreprocessor.from_preset(
    "bert_tiny_en_uncased",
    sequence_length=512,
)

In [25]:
train_cached = (
    train_ds.map(preprocessor, tensorflow.data.AUTOTUNE).cache().prefetch(tensorflow.data.AUTOTUNE)
)

In [26]:
test_cached = (
    test_ds.map(preprocessor, tensorflow.data.AUTOTUNE).cache().prefetch(tensorflow.data.AUTOTUNE)
)

In [27]:
custom_model = keras_nlp.models.BertClassifier.from_preset(
    "bert_tiny_en_uncased", preprocessor=None, num_classes=2
)
custom_model.fit(
    train_cached.batch(10),
    validation_data=test_cached.batch(10),
    epochs=3,
)

Epoch 1/3
4000/4000 ━━━━━━━━━━━━━━━━━━━━ 55s 9ms/step - loss: 0.4556 - sparse_categorical_accuracy: 0.7662 - val_loss: 0.2856 - val_sparse_categorical_accuracy: 0.8811
Epoch 2/3
4000/4000 ━━━━━━━━━━━━━━━━━━━━ 34s 8ms/step - loss: 0.2524 - sparse_categorical_accuracy: 0.8978 - val_loss: 0.2610 - val_sparse_categorical_accuracy: 0.8954
Epoch 3/3
4000/4000 ━━━━━━━━━━━━━━━━━━━━ 34s 8ms/step - loss: 0.1859 - sparse_categorical_accuracy: 0.9281 - val_loss: 0.2936 - val_sparse_categorical_accuracy: 0.8909


In [28]:
predictions = custom_model.predict(test_cached.batch(batch_size=10))

1000/1000 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step


In [29]:
preprocessing_sentiment_preds = [classifier_predict(pred) for pred in predictions]

In [30]:
preprocessing_sentiment_preds

[1,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 1,
 0,
 0,
 0,
 1,
 1,
 0,
 1,
 0,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 1,
 1,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 0,
 1,
 0,
